In [1]:
import pymorphy2
import pandas as pd
from collections import Counter
import requests
from bs4 import BeautifulSoup

In [2]:
# Список ключевых слов
keywords = [
    "Коррупция", "Взятки", "Подкуп", "Злоупотребление полномочиями", "Непотизм",
    "Конфликт интересов", "Теневая экономика", "Финансовые махинации", "Противодействие коррупции", 
    "Транспарентность", "Подотчетность", "Мониторинг", "Антикоррупционная политика",
    "Прозрачность финансирования", "Теневые сделки", "Откаты", "Хищение",
    "Лоббизм", "Незаконное обогащение", "Контроль расходов", "Проверки деятельности",
    "Аудит", "Социальный контроль", "Общественные расследования", "Антикор",
    "Мониторинг закупок", "Эффективность управления", "Ответственность руководства", "Правозащитные организации",
    "Уголовное преследование", "Электронное правительство", "Похищение", "Отмывание", "Задержание", "Рейдерство",
    "Превышение власти"
]

keywords_english = [
    "Corruption", "Bribes", "Kickbacks", "Abuse of authority", "Nepotism",
    "Conflict of interest", "Shadow economy", "Financial machinations", "Anti-corruption efforts", 
    "Transparency", "Accountability", "Monitoring", "Anti-corruption policy",
    "Financial transparency", "Shadow deals", "Embezzlement", "Theft",
    "Lobbying", "Illicit enrichment", "Expenditure control", "Audits",
    "Audit", "Social control", "Public investigations", "Anti-corruption",
    "Procurement monitoring", "Management efficiency", "Responsibility of leadership", 
    "Human rights organizations", "Criminal prosecution", "E-Government", 
    "Abduction", "Money laundering", "Arrest", "Raiding", "Abuse of power"
]

In [3]:
dataset = pd.DataFrame(columns=["Date"] + keywords_english)
# Инициализация pymorphy2
morph = pymorphy2.MorphAnalyzer()

# Функция для генерации всех возможных форм слова
def generate_forms(word):
    parsed = morph.parse(word)[0]
    return {form.word for form in parsed.lexeme}

# Генерируем все формы для ключевых слов
keyword_forms = {}
for keyword in keywords:
    keyword_forms[keyword.lower()] = generate_forms(keyword.lower())

# Объединяем все формы в один набор для поиска
all_keyword_forms = set()
for forms in keyword_forms.values():
    all_keyword_forms.update(forms)

# Лемматизация текста и поиск ключевых слов
def find_keywords_in_text(text):
    tokens = text.lower().split()  # Разбиваем текст на слова
    matched_keywords = [token for token in tokens if token in all_keyword_forms]
    return Counter(matched_keywords)

AttributeError: module 'inspect' has no attribute 'getargspec'

In [ ]:
url = "https://tengrinews.kz/tag/%D0%BA%D0%BE%D1%80%D1%80%D1%83%D0%BF%D1%86%D0%B8%D1%8F/"
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

In [ ]:
news_list = soup.find_all('div', {'class': 'content_main_item'})
stop_point = 2  # Лимит ссылок для тестирования
news_links = []

for news_item in news_list:
    first_link = news_item.find('a', href=True)  # Берём только первую ссылку внутри каждого блока
    if first_link:  # Фильтруем по началу ссылки
        news_links.append(first_link['href'])
    # if len(news_links) >= stop_point:  # Ограничиваем количество ссылок
    #     break
        
# Убираем дублирующиеся ссылки
news_links = list(set(news_links))

In [ ]:
news_links

In [ ]:
stop_point = 2  # Лимит ссылок для тестирования

# Парсим новости
for link in news_links:
    # Проверяем, является ли ссылка абсолютной
    if link.startswith('https'):
        full_url = link  # Абсолютную ссылку оставляем без изменений
    else:
        full_url = f"https://tengrinews.kz{link}"  # Добавляем домен только к относительным ссылкам
    
    # Получаем текст новости
    news_response = requests.get(full_url, headers=headers)
    news_soup = BeautifulSoup(news_response.text, "html.parser")
    
    # Извлекаем дату публикации
    date_element = news_soup.find('div', {'class': 'date-time'})
    if date_element:
        date_full = date_element.get_text(strip=True)
        date = date_full.split('|')[0].strip()  # Берём только день, месяц и год

    # Извлекаем текст новости
    article_body = news_soup.find('div', {'class': 'content_main_text'})
    if article_body:
        text = article_body.get_text(strip=True)
        keyword_counts = find_keywords_in_text(text)

        # Создаём словарь для строки DataFrame
        row = {key: 0 for key in keywords_english}
        row["Date"] = date


        # Заполняем количество упоминаний ключевых слов
        for keyword_rus, count in keyword_counts.items():
            keyword_eng = keywords_english[keywords.index(keyword_rus.capitalize())]
            row[keyword_eng] = count

        # Добавляем строку в DataFrame
        dataset = pd.concat([dataset, pd.DataFrame([row])], ignore_index=True)

        # # Выводим результат
        # print(f"URL: {full_url}")
        # print("Ключевые слова и количество упоминаний:")
        # for keyword, count in keyword_counts.items():
        #     if count > 0:
        #         print(f"{keyword}: {count}")
        # print("-" * 40)

        print(article_body.text)
    # if len(news_links) >= stop_point:  # Ограничиваем количество ссылок
    #     break


In [ ]:
# Сохраняем DataFrame в файл
dataset.to_csv("news_dataset.csv", index=False)

# Просмотр результата
print(dataset.head())

In [ ]:
# # news_response = requests.get("https://tengrinews.kz/sng/spetsslujbyi-zaderjali-eks-glavu-mchs-kyirgyizstana-556711/", headers=headers)
# # news_soup = BeautifulSoup(news_response.text, "html.parser")

# # article_body = news_soup.find('div', {'class': 'content_main_text'})

# text = article_body.get_text(strip=True)
# keyword_counts = find_keywords_in_text(text)

# # Выводим результат
# print(f"URL: {full_url}")
# print("Ключевые слова и количество упоминаний:")
# for keyword, count in keyword_counts.items():
#     if count > 0:
#         print(f"{keyword}: {count}")
# print("-" * 40)
